# ASEを使ってQE用の構造モデル(金属スラブ、分子吸着モデル）を作成する

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
import os, copy
import numpy as np
import nglview as nv
import matplotlib.pyplot as plt
import qeutil
from qeutil import extract_structure_data, write_qe_input
from ase import Atoms
from ase.io import read, write
from ase.data import atomic_masses, atomic_numbers
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.constraints import FixAtoms
from ase.io.cube import read_cube_data
from ase.build import fcc111, graphene, add_adsorbate, molecule
from ase.geometry import wrap_positions

## Slabモデルを作成する

ASEにはあらかじめ典型的な表面構造を作る関数が用意されている。用意されている構造をリストする。詳しくは[ase.build.surface](https://wiki.fysik.dtu.dk/ase/_modules/ase/build/surface.html)を参照。

||||||||
|---|---|---|---|---|---|---| 
|Surface|fcc(100)|fcc(110)|fcc(111)|bcc(100)|bcc(110)|bcc(111)|
|Adsorption site|ontop, bridge<br>hollow|ontop, longbridge<br>shortbridge, hollow|ontop, bridge<br>fcc, hcp|ontop, bridge<br>hollow|ontop, longbridge<br>shortbridge, hollow|ontop|
|Surface|hcp(0001)|hcp(10$\bar{1}$0)|diamond(111)|diamond(100)|mx2|graphene|
|Adsorption site|ontop, bridge<br>fcc, hcp|ontop|ontop|ontop|||

### 吸着サイトの情報を見る

吸着サイトの指定の仕方は二通りある：```ontop```などのキーワードを与える方法と、$(x,y)$を直接与える方法である。各表面構造関数が持っている吸着サイトは以下のように調べることができる。例として```fcc(111)```表面に用意されている吸着サイトを表示してみる。

In [ ]:
slab = fcc111('Al', size=(1,1,5), vacuum=10.0)
slab.info.get('adsorbate_info', {})

このように、```ontop, bridge, fcc```および```hcp```サイトがあることが分かる。もし、あらかじめ用意されている吸着サイトがない場合は何も表示されない。

In [ ]:
slab = graphene(formula='C2', size=(1,1,1), vacuum=10.0)
slab.info.get('adsorbate_info', {})

このような場合は、直接$xy$座標を指定して吸着サイトを指定する必要がある。（詳しくは下で説明します。）

### fcc(111)表面を作る

fcc111関数を用いて表面を作成する。```size```で表面の周期とスラブの厚みを設定できる。左右の真空の厚みはそれぞれ```lvac```と```rvac```で指定できる。

In [ ]:
lvac = 8.0 # 左側の真空の厚み
rvac = 10.0 # 右側の真空の厚み
vac = (lvac + rvac)/2.0 # 表面構造関数には真空の左右の真空の厚みの半分を渡す
slab = fcc111('Al', size=(4,4,2), vacuum=vac)
slab.wrap() # 普通に作ると、ユニットセルをはみ出す原子があるので、ユニットセル内にwrapする。
slab.translate((0.0,0.0,(lvac-rvac)/2.0)) # lvac, rvacを反映した位置にずらす
fig, ax = plt.subplots(1,2, figsize=(12, 6))
ax[0].set_axis_off()
ax[1].set_axis_off()
# 二次元投影図を作成
plot_atoms(slab, ax[0], radii=1.2, rotation=('0x,0y,0z'))
plot_atoms(slab, ax[1], radii=1.2, rotation=('90x,90y,90z'))
plt.show()
# NGLViewer を作成
#view_obj = nv.show_ase(slab)
#view_obj.clear_representations() # 既存の表現をクリア
#view_obj.add_ball_and_stick(radius=1.0) # Ball & Stick 表示を設定 (radius=1.0)
#view_obj.add_unitcell() # ユニットセルを表示
#view_obj.camera = 'orthographic' # 視点を正投影 (orthographic) に変更
#view_obj.stage.set_parameters(fogNear=100000, fogFar=1000000)  # Depth Cueing（遠くの色が薄くなる効果）を完全に無効化
#view_obj.parameters = { # クリッピング設定を調整
#    "clipNear": -100,  # クリッピング範囲を拡大
#    "clipFar": 100,   # 遠くのオブジェクトが消えないようにする
#    "clipDist": 0     # カメラとオブジェクトのクリッピングを最小に
#}
#view_obj # 表示
#view(slab, viewer='ngl')

スラブが意図した通り作成できているかを```slab```の中身を見て確認する。

In [ ]:
aobj = slab
print(f' Unit cell: a = ({aobj.cell[0,0]:9.5f}, {aobj.cell[0,1]:9.5f}, {aobj.cell[0,2]:9.5f})')
print(f'            b = ({aobj.cell[1,0]:9.5f}, {aobj.cell[1,1]:9.5f}, {aobj.cell[1,2]:9.5f})')
print(f'            c = ({aobj.cell[2,0]:9.5f}, {aobj.cell[2,1]:9.5f}, {aobj.cell[2,2]:9.5f})')
print(f' Number of atoms: {len(aobj.positions):5d}')
print(f' Species, Positions:')
for i in range(len(aobj.positions)):
    print(f'  \'{aobj.symbols[i]:<2}\' ({aobj.positions[i,0]:9.5f}, {aobj.positions[i,1]:9.5f}, {aobj.positions[i,2]:9.5f})')

### 分子吸着を考えるために分子を用意する

二酸化炭素を作成してみる。

In [ ]:
atoms_co2 = molecule('CO2')
fig, ax = plt.subplots(1,1, figsize=(3,1.5))
ax.set_axis_off()
plot_atoms(atoms_co2, ax, radii=1.0, rotation=('0x,90y,0z'))
plt.show()
#view(atoms_co2, viewer='ngl')

座標を確認する

In [ ]:
aobj = atoms_co2
print(f' Number of atoms: {len(aobj.positions):5d}')
print(f' Species, Positions:')
for i in range(len(aobj.positions)):
    print(f'  \'{aobj.symbols[i]:<2}\' ({aobj.positions[i,0]:9.5f}, {aobj.positions[i,1]:9.5f}, {aobj.positions[i,2]:9.5f})')

水分子を作成してみる。

In [ ]:
atoms_h2o = molecule('H2O')
fig, ax = plt.subplots(1,1, figsize=(3,1.5))
ax.set_axis_off()
plot_atoms(atoms_h2o, ax, radii=1.0, rotation=('0x,0y,0z'))
plt.show()

座標を確認する

In [ ]:
aobj = atoms_h2o
print(f' Number of atoms: {len(aobj.positions):5d}')
print(f' Species, Positions:')
for i in range(len(aobj.positions)):
    print(f'  \'{aobj.symbols[i]:<2}\' ({aobj.positions[i,0]:9.5f}, {aobj.positions[i,1]:9.5f}, {aobj.positions[i,2]:9.5f})')

### 分子を表面に置く

上で作った```slab```に水分子と二酸化炭素を置いてみる。まずは```fcc111```が持っている```ontop```サイトに水分子を酸素が下向きになるように置いてみる。分子を置くには```add_adsorbate```関数を使う。引数の意味は以下の通り(より詳しくは[ase.build.surface](https://wiki.fysik.dtu.dk/ase/_modules/ase/build/surface.html)を参照):<BR>
```height```: 吸着分子の高さを指定<BR>
```position```: 吸着位置を指定。```ontop```などのキーワードでも、直接$xy$座標を与えても良い<BR>
```offset```: 吸着分子をずらす。表面の単位格子に対してオフセットさせる。整数値を与えると```position```で与えたサイト上に置かれるが、1/2などで別の吸着サイトへオフセットさせることも可能。<BR>
```mol_index```: 吸着分子の```mol_index```番目の原子が吸着サイトの真上に来るように微調整する。

In [ ]:
# スラブを作成
lvac = 8.0 # 左側の真空の厚み
rvac = 15.0 # 右側の真空の厚み
vac = (lvac + rvac)/2.0 # 表面構造関数には真空の左右の真空の厚みの半分を渡す
slab = fcc111('Al', size=(4,4,2), vacuum=vac)
slab.wrap() # 普通に作ると、ユニットセルをはみ出す原子があるので、ユニットセル内にwrapする。
slab.translate((0.0,0.0,(lvac-rvac)/2.0)) # lvac, rvacを反映した位置にずらす
atoms_h2o.rotate(180, 'x', center=(0,0,0)) # 酸素が下になるように回転させる。
# ```ontop```サイトの上に水分子を置く
add_adsorbate(slab,atoms_h2o,height=5.0, position='ontop', offset=(0, 0), mol_index=0)
add_adsorbate(slab,atoms_co2,height=5.0, position='ontop', offset=(1, 2), mol_index=0)
fig, ax = plt.subplots(2,1, figsize=(12, 6))
ax[0].set_axis_off()
ax[1].set_axis_off()
plot_atoms(slab, ax[0], radii=1.2, rotation=('0x,0y,0z'))
plot_atoms(slab, ax[1], radii=1.2, rotation=('90x,90y,90z'))
plt.show()

### ESM/ESM-RISM計算用にスラブをずらす

上のスラブの例ではユニットセルが、$z=0$から$z=L_z$だと思って原子を配置している。ここで、$L_z$は$z$軸方向のユニットセルの大きさを表す。ESMやESM-RISM計算ではユニットセルの位置が$-L_z/2$から$L_z/2$として原子を配置する必要がある。そこで、ESM/ESM-RISM計算用のインプットを作るために、原子位置を$L_z/2$だけ左にずらす。

In [ ]:
slab_ESM = copy.deepcopy(slab)
slab_ESM.translate((0.0,0.0,-slab.cell[2,2]/2.0)) # shift atoms to fit ESM/ESM-RISM model
#fig, ax = plt.subplots(2,1, figsize=(12, 6))
#ax[0].set_axis_off()
#ax[1].set_axis_off()
#plot_atoms(slab_ESM, ax[0], radii=1.0, rotation=('0x,0y,0z'))
#plot_atoms(slab_ESM, ax[1], radii=1.0, rotation=('90x,90y,90z'))
#plt.show()

原子座標などを確認する。ここで、確認することは、
- スラブの左端の原子の$z$座標が$-L_z/2+$ ```lvac```となっているか。
- スラブの右端の原子の$z$座標が$L_z/2-$ ```rvac```となっているか。
- 分子を吸着している場合は、吸着分子の右端の原子から$L_z/2$までに十分な真空領域が確保されているか。（最低でも8Aは必要）

In [ ]:
aobj = slab_ESM
print(f' Unit cell: a = ({aobj.cell[0,0]:9.5f}, {aobj.cell[0,1]:9.5f}, {aobj.cell[0,2]:9.5f})')
print(f'            b = ({aobj.cell[1,0]:9.5f}, {aobj.cell[1,1]:9.5f}, {aobj.cell[1,2]:9.5f})')
print(f'            c = ({aobj.cell[2,0]:9.5f}, {aobj.cell[2,1]:9.5f}, {aobj.cell[2,2]:9.5f})')
print(f' Number of atoms: {len(aobj.positions):5d}')
print(f' Species, Positions:')
for i in range(len(aobj.positions)):
    print(f'  \'{aobj.symbols[i]:<2}\' ({aobj.positions[i,0]:9.5f}, {aobj.positions[i,1]:9.5f}, {aobj.positions[i,2]:9.5f})')

# ESM用のインプットファイルを作成する。

In [ ]:
input_data = {
    'control': {
        'calculation': 'scf',
        'restart_mode': 'from_scratch',
        'prefix': 'Al100_bc1',
        'disk_io': 'low',
        'lfcp': True,
        'trism': True,
    },
    'system': {
        'ibrav': 0,
        'ecutwfc': 20,
        'ecutrho': 160,
        'occupations' : 'smearing',
        'smearing':'mp',
        'degauss' : 0.03,
        'assume_isolated': 'esm',
        'esm_bc': 'bc1'
    },
    'electrons': {
        'mixing_beta': 0.3,
    },
    'fcp': {
        'fcp_mu': -3.5
    }
}
structure_data = extract_structure_data(slab_ESM)
#pseudopotentials = {'Al':'Al.pbe-n-van.UPF'}
pseudopotentials = {
    'Al': 'Al.pbe-n-van.UPF',
    'H': 'H.pbe-rrkjus.UPF',
    'C': 'C.pbe-rrkjus.UPF',
    'O': 'O.pbe-n-rrkjus.UPF'
}
atomic_species = {
    elem: (atomic_masses[atomic_numbers[elem]], pseudopotentials[elem])
    for elem in pseudopotentials
}
inpfile = 'test.in'
write_qe_input(input_data, atomic_species, structure_data, filename=inpfile, kpts=(16, 16, 1), koffset=(1, 1, 0)) 
ciffile = os.path.splitext(inpfile)[0] + ".cif" # CIF ファイル名を `inpfile` から生成
write(ciffile, slab) # CIF ファイルを出力(ESM用のshiftをしていない構造）
print(f"CIF file '{ciffile}' has been generated.")